# Fine tune GPT-4o-mini for classification


In [7]:
!pip install openai --quiet

In [47]:
SYSTEM_PROMPT = """You are a classifier. Given a question, you need to classify it into one of the following classes:\n - CANCEL\n - INVOICE\n - DELIVERY\n - FEEDBACK\n - CONTACT\n - ORDER\n - REFUND\n - SHIPPING\n - SUBSCRIPTION\n - PAYMENT\n - ACCOUNT\n\nDo not try to answer the question."""

In [48]:
# read train.csv and loop over the rows and take the question column, and the class_name column
# and create a list of objects that look like this:
# {
#         "messages": [
#             {"role": "system", "content":SYSTEM_PROMPT},
#             {"role": "user", "content": << column question>>},
#             {"role": "assistant",
#              "content": "<< column class_name >>"}
#         ]
# }
import pandas as pd
df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')

def convert_to_messages(df):
    data = []
    for i, row in df.iterrows():
        data.append({
            "messages": [
                {"role": "system", "content":SYSTEM_PROMPT},
                {"role": "user", "content": row['question']},
                {"role": "assistant",
                 "content": row['class_name']}
            ]
        })
    return data

data_train = convert_to_messages(df_train)
data_val = convert_to_messages(df_val)
    

In [50]:
import json

def to_openai_format(data, file_path):
    with open(
            file_path, "w", encoding="utf-8"
    ) as file:
        for record in data:
            json_line = json.dumps(record)
            file.write(json_line + "\n")

TRAIN_OPENAI_FILE = "train-openai.jsonl"
VAL_OPENAI_FILE = "val-openai.jsonl"
to_openai_format(data=data_train, file_path=TRAIN_OPENAI_FILE)
to_openai_format(data=data_val, file_path=VAL_OPENAI_FILE)


In [51]:
import openai

client = openai.OpenAI()

def create_file(file_path):
    response = client.files.create(
        file=open(file_path, "rb"),
        purpose="fine-tune"
    )
    return response.id

train_id = create_file(file_path=TRAIN_OPENAI_FILE)
val_id = create_file(file_path=VAL_OPENAI_FILE)

response = client.fine_tuning.jobs.create(
    training_file=train_id,
    validation_file=val_id,
    model="gpt-4o-mini-2024-07-18"
)
response


FineTuningJob(id='ftjob-uchj4y2eWRV7OjUuGZztUzR1', created_at=1725703100, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-bBHdSNHiFHSbkdpwXvV7ClOQ', result_files=[], seed=100034588, status='validating_files', trained_tokens=None, training_file='file-4cf9pcZ43oaKMqMP33oYijqw', validation_file='file-d6Xxrqztbi1JjAYsX7JLWJzY', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [61]:

MODEL_ID = "ft:gpt-4o-mini-2024-07-18:drift::A4nSvjUg"

# Function to make a request to your fine-tuned model
def call_finetuned_model(prompt):
    try:
        # Call the completion endpoint with your fine-tuned model
        response = client.chat.completions.create(
            model=MODEL_ID,  # Use the fine-tuned model ID
            messages=[    
                {"role":"system", "content": SYSTEM_PROMPT},
                {"role":"user", "content": prompt}
            ]
        )
        # Extract and return the generated text
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Error: {str(e)}"



In [ ]:
# for each row in the test.csv file, take the question column and call the function call_finetuned_model
# with the question as input and store the output in a new column called predicted_class_name
df_test = pd.read_csv('test.csv')
df_test['predicted_class_name'] = df_test['question'].apply(call_finetuned_model)
df_test

In [ ]:
def calculate_accuracy(df):
    # use the huggingface evaluate library to evaluate the model by taking the columns
    # 'predicted_class_name' and 'class_name' as input andd calculate the accuracy
    from datasets import load_metric
    # Create a mapping of class names to numerical labels
    unique_classes = set(df['predicted_class_name']).union(set(df['class_name']))
    class_to_int = {cls_name: idx for idx, cls_name in enumerate(unique_classes)}
    
    # Map the class names to integers
    df['predicted_class_numeric'] = df['predicted_class_name'].map(class_to_int)
    df['class_numeric'] = df['class_name'].map(class_to_int)
    
    # Compute the accuracy
    metric = load_metric("accuracy")
    accuracy = metric.compute(predictions=df['predicted_class_numeric'], references=df['class_numeric'])
    
    print(f"Accuracy: {accuracy['accuracy']}")
    return accuracy

calculate_accuracy(df_test)